In [1]:
import pandas as pd
import tester
from trainer import read_tables
import os
import argparse
import json
from IPython.display import display, clear_output
import time


/home/cc/code/pyenv/fusion_decoder/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_args(dataset):
    work_dir=os.path.abspath('..')
    test_args = argparse.Namespace(work_dir=work_dir,
                                   query_dir='demo_query',
                                   dataset=dataset)
    return test_args

In [3]:
def create_query_file(data_dir, question):
    file_name = 'fusion_query.jsonl'
    data_file = os.path.join(data_dir, file_name)
    query_data = {
        "id":0,
        "question":question,
        "table_id_lst":['N/A'],
        "answers":["N/A"],
        "ctxs": [{"title": "", "text": "This is a example passage."}]
    }
    with open(data_file, 'w') as f_o:
        f_o.write(json.dumps(query_data) + '\n')
    

In [4]:
def show_results(question, out_dir, table_dict):
    data_file = os.path.join(out_dir, 'pred_None.jsonl')
    with open(data_file) as f:
        item = json.load(f)
    tag_lst = item['tags']
    N = 5
    pred_table_set = set()
    for tag in tag_lst:
        table_id = tag['table_id']
        if table_id not in pred_table_set:
            pred_table_set.add(table_id)
            table = table_dict[table_id]
            show_table(question, table, len(pred_table_set))
        if len(pred_table_set) >= 5:
            return
    
def show_table(question, table, seq_no):
    print("Question, %s" % question)
    print("Top %d table,         %s " % (seq_no, table['documentTitle']))
    col_lst = table['columns']
    col_names = [a['text'] for a in col_lst]
    row_lst = table['rows']
    row_data = []
    for row_item in row_lst:
        cell_lst = row_item['cells']
        row_item = [a['text'] for a in cell_lst]
        row_data.append(row_item)
    df = pd.DataFrame(row_data,columns=col_names)
    display(df)
    

In [ ]:
dataset=input('Type the dataset, ')
args = get_args(dataset)
print('Loading tables')
table_dict = read_tables(args.work_dir, dataset)
print('Loading index')
index_obj = tester.get_index_obj(args.work_dir, dataset)
data_dir = os.path.join(args.work_dir, 'data', dataset, 'demo_query', 'test')
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
question = input('Type a question (q to quit),\n').strip()
while question != 'q':
    create_query_file(data_dir, question)
    out_dir = tester.main(args, table_data=table_dict, index_obj=index_obj)
    clear_output(wait=False)
    show_results(question, out_dir, table_dict)
    time.sleep(1)
    question = input('Type a question (q to quit),\n').strip()
    

Question, What and when were Tom Felton's first two roles?
Top 1 table,         Tom Felton , Films 


,Year,Title,Role,Notes
0,1997,The Borrowers,Peagreen Clock,-
1,1999,Anna and the King,Louis T. Leonowens,-
2,2001,Harry Potter and the Philosopher's Stone,Draco Malfoy,Released as Harry Potter and the Sorcerer's St...
3,2002,Harry Potter and the Chamber of Secrets,Draco Malfoy,-
4,2004,Harry Potter and the Prisoner of Azkaban,Draco Malfoy,-
5,2005,Harry Potter and the Goblet of Fire,Draco Malfoy,-
6,2007,Harry Potter and the Order of the Phoenix,Draco Malfoy,-
7,2008,The Disappeared,Simon Pryor,-
8,2009,Harry Potter and the Half-Blood Prince,Draco Malfoy,-
9,2010,Harry Potter and the Deathly Hallows – Part 1,Draco Malfoy,-


Question, What and when were Tom Felton's first two roles?
Top 2 table,         Tom Felton , Awards and nominations 


,Year,Work,Award,Category,Result,Refs
0,2001,Harry Potter and the Philosopher's Stone,Young Artist Award,Best Ensemble in a Feature Film,Nominated,-
1,2001,Harry Potter and the Philosopher's Stone,Young Artist Award,Best Performance in a Feature Film: Supporting...,Nominated,-
2,2009,Harry Potter and the Half-Blood Prince,MTV Movie Award,Best Villain,Won,-
3,2009,Harry Potter and the Half-Blood Prince,Scream Award,Best Ensemble,Won,-
4,2010,Harry Potter and the Deathly Hallows – Part 1,MTV Movie Award,Best Villain,Won,-
5,2010,Harry Potter and the Deathly Hallows – Part 1,Teen Choice Award,Choice Movie Villain,Won,-
6,2011,Harry Potter and the Deathly Hallows – Part 2,MTV Movie Award,Best Cast,Won,-
7,2011,Harry Potter and the Deathly Hallows – Part 2,Scream Award,Best Ensemble,Nominated,-
8,2015,Himself,Giffoni Film Festival,Giffoni Experience Award,Won,-


Question, What and when were Tom Felton's first two roles?
Top 3 table,         Thomas Gibson , Television 


,Year,Title,Role,Notes
0,1987,Leg Work,Robbie,"Episode: ""All This and a Gold Card Too"""
1,1987,Guiding Light,Peter Latham,-
2,1988,Lincoln,William Sprague,Miniseries
3,1988–1990,As the World Turns,Derek Mason,-
4,1990,The Kennedys of Massachusetts,Peter Fitzwilliam,Miniseries
5,1990,Another World,Sam Fowler,-
6,1993,Tales of the City,Beauchamp Day,Miniseries
7,1994–1997,Chicago Hope,Daniel Nyland,70 episodes
8,1995,Secrets,Hailus Tuckman,Television film
9,1996,Night Visitors,Ross Williams,Television film


Question, What and when were Tom Felton's first two roles?
Top 4 table,         Tom McGowan , Movie credits 


,Film,Release Date,Role
0,Play Nice,2006,-
1,12 and Holding,2005,Patrick Fisher
2,After the Sunset,2004,Ed
3,Dog Gone Love,2004,Star
4,Bad Santa,2003,Harrison
5,Ghost World,2001,Joe
6,The Family Man,2000,Bill
7,True Crime,1999,Tom Donaldson
8,As Good as It Gets,1997,Maitre D'
9,Bean: The Movie,1997,Walter Huntley


Question, What and when were Tom Felton's first two roles?
Top 5 table,         Tom Baker , Film 


,Year,Title,Role,Notes
0,1967,The Winter's Tale,-,-
1,1971,Nicholas and Alexandra,Rasputin,-
2,1972,The Canterbury Tales,Jenkin,-
3,1973,Cari Genitori,Karl,-
4,1973,The Vault of Horror,Moore,-
5,1973,Luther,Pope Leo X,Doesn't appear in some versions of the film
6,1973,Frankenstein: The True Story,Sea captain,-
7,1973,The Golden Voyage of Sinbad,Koura,-
8,1974,The Mutations,Lynch,-
9,1980,The Curse of King Tut's Tomb,Hasan,-


Example questions in fetaqa dataset,

1. What shows did Marisol Nichols appear in during 1996?

2. What and when were Tom Felton's first two roles?

3. How close was Texas' 21st congressional district election?